## Setup

In [ ]:
# Load packages
using DifferentialEquations
using Plots, StatsPlots
using CSV, DataFrames
using Turing
using LaTeXStrings
using XLSX
using Optim
using StatsBase
using Random
using KernelDensity
using ProgressMeter
using Distributions
using Measures
using GLM

# Setup plots standard
Plots.default(fontfamily = ("computer modern"))

# Set seed 
Random.seed!(42)

In [ ]:
# Add rhs file
include("model_rhs.jl")
    
# Add parameter file
include("model_default_param.jl")

# Add treatment rhs
include("model_rhs_treatment.jl")

# Add function for calculating VAF
include("model_calc_VAF.jl")

# Add function for calculating VAF - alpha
include("model_calc_VAF_alpha.jl")

# Add model for Turing inference - individual
include("model_infer_dynamics.jl")

# Add model for Turing inference - individual - alpha
include("model_infer_dynamics_alpha.jl")

# Add model function for Turing inference - individual using prior knowledge
include("model_infer_dynamics_post_Hierarchical.jl")

# Add model function for Turing inference - individual using prior knowledge - alpha
include("model_infer_dynamics_post_Hierarchical_alpha.jl")

In [ ]:
# Setup initial conditions for master curve
x00 = 1.0*10^5
x10 = 2.5*10^6
x20 = 6.4*10^11
y00 = 1
y10 = 0
y20 = 0
a0 = 8.1*10^2
s0 = 1

# Collect in one vector
u0 = [x00,x10,x20,y00,y10,y20,a0,s0]

# Setup and solve ODEproblem
tspan = (0.0,90*365)
prob = ODEProblem(model_rhs, u0, tspan, p)
sol = solve(prob, TRBDF2(), reltol = 1e-10, abstol = 1e-10, saveat=1)

# Save as named tuple
master_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))

## Data

In [ ]:
# Setup dataframes
df_D = DataFrame

# Load data
path = "M:/data_cancer/DALIAH/Trine_DALIAH_5y/DALIAH_5y - Export for Julia.csv"
df_D = CSV.read(path, df_D)

# Change data types
df_D[!,:days] = convert.(Float64,df_D[:,:days])
df_D[!,:RUX] = convert.(Float64,df_D[:,:RUX])


df_D

In [ ]:
# Convert to starting at time for first data point - no matter if JAK or treatment
for i in unique(df_D.patientID)
    # Set pID
    pID = i
    
    # Extract minimum number of days - where IFN>0 or JAK>=0
    df_p = df_D[df_D.patientID .== pID,:]

    minDays = minimum(df_p.days[df_p.IFN .> 0  .|| df_p.JAK .>= 0, :])
    
    # Subtract minimum number of days
    df_D.days[df_D.patientID .== pID, :] = df_D.days[df_D.patientID .== pID, :] .- minDays
end
df_D

## Calculate and plot a single patients

In [ ]:
pID = 3
df_p = df_D[df_D.patientID .== pID, :]

# Extract latest data point
maxDays = maximum(df_p.days)

# Extract VAF
pVAF = df_p.JAK

# Use only data at which VAF is not NaN
pVAF = pVAF[.!isnan.(pVAF)]


# Choose treatment effect
effect = "py0dy1"

# Chooase to estimate initJAK as well
estInitJAK = true
    
# Instantiate model
model = model_infer_dynamics_alpha(pVAF,df_p,effect,p,master_curve,estInitJAK)

# Set seed 
Random.seed!(42)

# Sample using Turing
MCMC = sample(model, HMC(0.05, 10), 10500)

# Convert to data frame
df_MCMC_3 = DataFrame(MCMC)

In [ ]:
# Save results
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Alpha/MCMC_alpha_3.csv"
CSV.write(path, df_MCMC_3)

In [ ]:
# Extract data
pID = 3
df_p = df_D[df_D.patientID .== pID, :]

# Choose time for calculating VAF
pDays = 0:1:75*30.4
pDays2 = df_p.days[.!isnan.(df_p.JAK)]
pVAF2 = df_p.JAK[.!isnan.(df_p.JAK)]
pDays_plot = pDays[1:30:end]

# Extract latest data point
maxDays = maximum(df_p.days)

# Choose treatment effect
effect = "py0dy1"

# Load results
df_MCMC_3 = DataFrame
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Alpha/MCMC_alpha_3.csv"
df_MCMC_3 = CSV.read(path, df_MCMC_3)

# Remove burn-in
deleteat!(df_MCMC_3,collect(1:500))

 # Storage
VAF_mat = Array{Float64, 2}(undef, length(pDays), length(df_MCMC_3[:,"iteration"]))
VAF_mat_points = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC_3[:,"iteration"]))
p_mat = Array{Float64, 3}(undef, length(pDays), length(df_MCMC_3[:,"iteration"]), 8)

# Calculate VAF in loop
@showprogress dt=1 desc="Calculating VAF for MCMC chain..." for k in eachindex(df_MCMC_3.rho1)
    # Set rho values
    rho = [df_MCMC_3.rho1[k], df_MCMC_3.rho2[k], df_MCMC_3.alpha[k]]

    # Set initial JAK
    initJAK = df_MCMC_3.initJAK[k]

    # Calculate VAF using function
    VAF_mat[:,k], sol = model_calc_VAF_alpha(rho,df_p,effect,pDays,p,master_curve,initJAK)
    VAF_mat_points[:,k], sol_points = model_calc_VAF_alpha(rho,df_p,effect,pDays2,p,master_curve,initJAK)
    p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
               a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
    p_mat[:,k,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
end
VAF_mat_plot = VAF_mat[1:30:end,:]

In [ ]:
# Calculate RMSE based on mean at the points
RMSE = rmsd(median(VAF_mat_points, dims=2),df_p.JAK[.!isnan.(df_p.JAK)])

# Calculate mean width of 95% CI
mean_width = mean(abs.(quantile.(eachrow(VAF_mat), 0.975)*100-quantile.(eachrow(VAF_mat), 0.025)*100))

# Calculate mean parameter values
median_param_1 = median(df_MCMC_3[:,"rho1"])
median_param_2 = median(df_MCMC_3[:,"rho2"])
median_alpha = median(df_MCMC_3[:,"alpha"])

# Plot results
figVAF = plot()
xlims!(0, 75)
ylims!(0, 100)
# xlabel!(L"t"*" [months]")
# ylabel!(L"JAK2"*" VAF [%]")
annotate!(50, 90, text(L"\textrm{RMSE}=" * "$(round(RMSE; sigdigits=3))", :center, 15))
annotate!(50, 80, text(L"\rho="*"$(round(median_param_1; sigdigits=3))", :center, 15))
annotate!(50, 70, text(L"\delta="*"$(round(median_param_2; sigdigits=3))", :center, 15))
annotate!(50, 60, text(L"\alpha="*"$(round(median_alpha; sigdigits=3))", :center, 15))
title!("Patient $(pID)")
p2 = twinx()
xlims!(p2, 0, 75)
ylims!(p2, 0, 20)
plot!(p2, df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
        linewidth = 3, label = "", colour= :blue1)
plot!(p2, [maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
      linewidth = 3, label = "", colour= :blue1)
#         title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho="
#                *"$(round(median_param_1; sigdigits=3)), " * L"\delta="*"$(round(median_param_2; sigdigits=3))")
# ylabel!(p2, "IFN"*L"-$\alpha$ $\left[\mathrm{\mu g/day}\right]$")
plot!(pDays_plot/30.4, quantile.(eachrow(VAF_mat_plot), 0.025)*100, fillrange = quantile.(eachrow(VAF_mat_plot), 0.975)*100,
      fillalpha = 0.35, colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
plot!(pDays_plot[pDays_plot.<=maxDays]/30.4, median(VAF_mat_plot[pDays_plot.<=maxDays,:],dims=2)*100, linewidth = 3, 
  label = L"JAK2"*" VAF - Median", colour = :black)
plot!(pDays_plot[pDays_plot.>maxDays]/30.4, median(VAF_mat_plot[pDays_plot.>maxDays,:],dims=2)*100, linewidth = 3, 
  label = L"JAK2"*" VAF - Median prediction", colour = :black, linestyle = :dash)
scatter!(pDays2./30.4, pVAF2*100, label=L"\textrm{Data}", markercolor=:red, markersize=6, markerstrokewidth=0.12)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
figVAF_3 = plot(figVAF, size = (700, 500), margin=5mm, leftmargin = 8mm, rightmargin = 8mm)
xlims!(p2, 0, 75)
ylims!(p2, 0, 20)

In [ ]:
 # Plot density
l = @layout [a a; a a; a a; a a; a a;]
plot_rho1_chain = plot(df_MCMC_3.iteration,df_MCMC_3.rho1,linewidth = 0.5, colour = 1, label = "")
title!("\nChain for "*L"\rho")
xlabel!("Iteration")
ylabel!(L"\rho\quad \left[\mathrm{day/\mu g}\right]")
plot_rho1_density = plot(density(df_MCMC_3.rho1,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!("\nDensity for "*L"\rho")
xlabel!(L"\rho\quad \left[\mathrm{day/\mu g}\right]")
ylabel!("pdf")
plot_rho2_chain = plot(df_MCMC_3.iteration,df_MCMC_3.rho2,linewidth = 0.5, colour = 1, label = "")
title!("Chain for "*L"\delta")
xlabel!("Iteration")
ylabel!(L"\delta\quad\left[\mathrm{day/\mu g}\right]")
plot_rho2_density = plot(density(df_MCMC_3.rho2,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!("Density for "*L"\delta")
xlims!(0,1.1)
xlabel!(L"\delta\quad \left[\mathrm{day/\mu g}\right]")
ylabel!("pdf")
plot_alpha_chain = plot(df_MCMC_3.iteration,df_MCMC_3.alpha,linewidth = 0.5, colour = 1, label = "")
title!(L"Chain for $\alpha$")
xlabel!("Iteration")
ylabel!(L"$\alpha$")
plot_alpha_density = plot(density(df_MCMC_3.alpha,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!(L"Density for $\alpha$")
xlabel!(L"$\alpha$")
ylabel!("pdf")
plot_initJAK_chain = plot(df_MCMC_3.iteration,df_MCMC_3.initJAK*100,linewidth = 0.5, colour = 1, label = "")
title!(L"Chain for $v_0$")
xlabel!("Iteration")
ylabel!(L"$v_0$ [%]")
plot_initJAK_density = plot(density(df_MCMC_3.initJAK*100,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 100)
title!(L"Density for $v_0$")
xlabel!(L"$v_0$ [%]")
ylabel!("pdf")
plot_sigma_chain = plot(df_MCMC_3.iteration,df_MCMC_3.sigma*100, linewidth = 0.5, colour = 1, label = "")
title!("Chain for "*L"\sigma_v")
xlabel!("Iteration")
ylabel!(L"$\sigma_v$ [%]")
plot_sigma_density = plot(density((df_MCMC_3.sigma*100),legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 60)
title!("Density for "*L"\sigma_v")
xlims!(0,100)
xlabel!(L"$\sigma_v$ [%]")
ylabel!("pdf")
figMCMC_3 = plot(plot_rho1_chain,plot_rho1_density,plot_rho2_chain,plot_rho2_density,plot_alpha_chain,plot_alpha_density,
    plot_initJAK_chain,plot_initJAK_density,
    plot_sigma_chain,plot_sigma_density,layout = l,size = (700, 1000), margin=5mm, leftmargin = 8mm, rightmargin = 8mm)
        

In [ ]:
pID = 38
df_p = df_D[df_D.patientID .== pID, :]

# Extract latest data point
maxDays = maximum(df_p.days)

# Extract VAF
pVAF = df_p.JAK

# Use only data at which VAF is not NaN
pVAF = pVAF[.!isnan.(pVAF)]


# Choose treatment effect
effect = "py0dy1"

# Chooase to estimate initJAK as well
estInitJAK = true
    
# Instantiate model
model = model_infer_dynamics_alpha(pVAF,df_p,effect,p,master_curve,estInitJAK)

# Set seed 
Random.seed!(42)

# Sample using Turing
MCMC = sample(model, HMC(0.05, 10), 10500)

# Convert to data frame
df_MCMC_38 = DataFrame(MCMC)

In [ ]:
# Save results
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Alpha/MCMC_alpha_38.csv"
CSV.write(path, df_MCMC_38)

In [ ]:
# Extract data
pID = 38
df_p = df_D[df_D.patientID .== pID, :]

# Choose time for calculating VAF
pDays = 0:1:75*30.4
pDays2 = df_p.days[.!isnan.(df_p.JAK)]
pVAF2 = df_p.JAK[.!isnan.(df_p.JAK)]
pDays_plot = pDays[1:30:end]

# Extract latest data point
maxDays = maximum(df_p.days)

# Choose treatment effect
effect = "py0dy1"

# Load results
df_MCMC_38 = DataFrame
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Alpha/MCMC_alpha_38.csv"
df_MCMC_38 = CSV.read(path, df_MCMC_38)

# Remove burn-in
deleteat!(df_MCMC_38,collect(1:500))

 # Storage
VAF_mat = Array{Float64, 2}(undef, length(pDays), length(df_MCMC_38[:,"iteration"]))
VAF_mat_points = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC_38[:,"iteration"]))
p_mat = Array{Float64, 3}(undef, length(pDays), length(df_MCMC_38[:,"iteration"]), 8)

# Calculate VAF in loop
@showprogress dt=1 desc="Calculating VAF for MCMC chain..." for k in eachindex(df_MCMC_38.rho1)
    # Set rho values
    rho = [df_MCMC_38.rho1[k], df_MCMC_38.rho2[k], df_MCMC_38.alpha[k]]

    # Set initial JAK
    initJAK = df_MCMC_38.initJAK[k]

    # Calculate VAF using function
    VAF_mat[:,k], sol = model_calc_VAF_alpha(rho,df_p,effect,pDays,p,master_curve,initJAK)
    VAF_mat_points[:,k], sol_points = model_calc_VAF_alpha(rho,df_p,effect,pDays2,p,master_curve,initJAK)
    p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
               a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
    p_mat[:,k,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
end
VAF_mat_plot = VAF_mat[1:30:end,:]

In [ ]:
# Calculate RMSE based on mean at the points
RMSE = rmsd(median(VAF_mat_points, dims=2),df_p.JAK[.!isnan.(df_p.JAK)])

# Calculate mean width of 95% CI
mean_width = mean(abs.(quantile.(eachrow(VAF_mat), 0.975)*100-quantile.(eachrow(VAF_mat), 0.025)*100))

# Calculate mean parameter values
median_param_1 = median(df_MCMC_38[:,"rho1"])
median_param_2 = median(df_MCMC_38[:,"rho2"])
median_alpha = median(df_MCMC_38[:,"alpha"])

# Plot results
figVAF = plot()
xlims!(0, 75)
ylims!(0, 100)
# xlabel!(L"t"*" [months]")
# ylabel!(L"JAK2"*" VAF [%]")
annotate!(50, 90, text(L"\textrm{RMSE}=" * "$(round(RMSE; sigdigits=3))", :center, 15))
annotate!(50, 80, text(L"\rho="*"$(round(median_param_1; sigdigits=3))", :center, 15))
annotate!(50, 70, text(L"\delta="*"$(round(median_param_2; sigdigits=3))", :center, 15))
annotate!(50, 60, text(L"\alpha="*"$(round(median_alpha; sigdigits=3))", :center, 15))
title!("Patient $(pID)")
p2 = twinx()
xlims!(p2, 0, 75)
ylims!(p2, 0, 20)
plot!(p2, df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
        linewidth = 3, label = "", colour= :blue1)
plot!(p2, [maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
      linewidth = 3, label = "", colour= :blue1)
#         title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho="
#                *"$(round(median_param_1; sigdigits=3)), " * L"\delta="*"$(round(median_param_2; sigdigits=3))")
# ylabel!(p2, "IFN"*L"-$\alpha$ $\left[\mathrm{\mu g/day}\right]$")
plot!(pDays_plot/30.4, quantile.(eachrow(VAF_mat_plot), 0.025)*100, fillrange = quantile.(eachrow(VAF_mat_plot), 0.975)*100,
      fillalpha = 0.35, colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
plot!(pDays_plot[pDays_plot.<=maxDays]/30.4, median(VAF_mat_plot[pDays_plot.<=maxDays,:],dims=2)*100, linewidth = 3, 
  label = L"JAK2"*" VAF - Median", colour = :black)
plot!(pDays_plot[pDays_plot.>maxDays]/30.4, median(VAF_mat_plot[pDays_plot.>maxDays,:],dims=2)*100, linewidth = 3, 
  label = L"JAK2"*" VAF - Median prediction", colour = :black, linestyle = :dash)
scatter!(pDays2./30.4, pVAF2*100, label=L"\textrm{Data}", markercolor=:red, markersize=6, markerstrokewidth=0.12)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
figVAF_38 = plot(figVAF, size = (700, 500), margin=5mm, leftmargin = 8mm, rightmargin = 8mm)
xlims!(p2, 0, 75)
ylims!(p2, 0, 20)

In [ ]:
 # Plot density
l = @layout [a a; a a; a a; a a; a a;]
plot_rho1_chain = plot(df_MCMC_38.iteration,df_MCMC_38.rho1,linewidth = 0.5, colour = 1, label = "")
title!("\nChain for "*L"\rho")
xlabel!("Iteration")
ylabel!(L"\rho\quad \left[\mathrm{day/\mu g}\right]")
plot_rho1_density = plot(density(df_MCMC_38.rho1,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!("\nDensity for "*L"\rho")
xlabel!(L"\rho\quad \left[\mathrm{day/\mu g}\right]")
ylabel!("pdf")
plot_rho2_chain = plot(df_MCMC_38.iteration,df_MCMC_38.rho2,linewidth = 0.5, colour = 1, label = "")
title!("Chain for "*L"\delta")
xlabel!("Iteration")
ylabel!(L"\delta\quad\left[\mathrm{day/\mu g}\right]")
plot_rho2_density = plot(density(df_MCMC_38.rho2,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!("Density for "*L"\delta")
xlims!(0,1.1)
xlabel!(L"\delta\quad \left[\mathrm{day/\mu g}\right]")
ylabel!("pdf")
plot_alpha_chain = plot(df_MCMC_38.iteration,df_MCMC_38.alpha,linewidth = 0.5, colour = 1, label = "")
title!(L"Chain for $\alpha$")
xlabel!("Iteration")
ylabel!(L"$\alpha$")
plot_alpha_density = plot(density(df_MCMC_38.alpha,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!(L"Density for $\alpha$")
xlabel!(L"$\alpha$")
ylabel!("pdf")
plot_initJAK_chain = plot(df_MCMC_38.iteration,df_MCMC_38.initJAK*100,linewidth = 0.5, colour = 1, label = "")
title!(L"Chain for $v_0$")
xlabel!("Iteration")
ylabel!(L"$v_0$ [%]")
plot_initJAK_density = plot(density(df_MCMC_38.initJAK*100,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 100)
title!(L"Density for $v_0$")
xlabel!(L"$v_0$ [%]")
ylabel!("pdf")
plot_sigma_chain = plot(df_MCMC_38.iteration,df_MCMC_38.sigma*100, linewidth = 0.5, colour = 1, label = "")
title!("Chain for "*L"\sigma_v")
xlabel!("Iteration")
ylabel!(L"$\sigma_v$ [%]")
plot_sigma_density = plot(density((df_MCMC_38.sigma*100),legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 60)
title!("Density for "*L"\sigma_v")
xlims!(0,100)
xlabel!(L"$\sigma_v$ [%]")
ylabel!("pdf")
figMCMC_38 = plot(plot_rho1_chain,plot_rho1_density,plot_rho2_chain,plot_rho2_density,plot_alpha_chain,plot_alpha_density,
    plot_initJAK_chain,plot_initJAK_density,
    plot_sigma_chain,plot_sigma_density,layout = l,size = (700, 1000), margin=5mm, leftmargin = 8mm, rightmargin = 8mm)
        

In [ ]:
pID = 85
df_p = df_D[df_D.patientID .== pID, :]

# Extract latest data point
maxDays = maximum(df_p.days)

# Extract VAF
pVAF = df_p.JAK

# Use only data at which VAF is not NaN
pVAF = pVAF[.!isnan.(pVAF)]


# Choose treatment effect
effect = "py0dy1"

# Chooase to estimate initJAK as well
estInitJAK = true
    
# Instantiate model
model = model_infer_dynamics_alpha(pVAF,df_p,effect,p,master_curve,estInitJAK)

# Set seed 
Random.seed!(42)

# Sample using Turing
MCMC = sample(model, HMC(0.05, 10), 10500)

# Convert to data frame
df_MCMC_85 = DataFrame(MCMC)

In [ ]:
# Save results
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Alpha/MCMC_alpha_85.csv"
CSV.write(path, df_MCMC_85)

In [ ]:
# Extract data
pID = 85
df_p = df_D[df_D.patientID .== pID, :]

# Choose time for calculating VAF
pDays = 0:1:75*30.4
pDays2 = df_p.days[.!isnan.(df_p.JAK)]
pVAF2 = df_p.JAK[.!isnan.(df_p.JAK)]
pDays_plot = pDays[1:30:end]

# Choose treatment effect
effect = "py0dy1"

# Extract latest data point
maxDays = maximum(df_p.days)

# Load results
df_MCMC_85 = DataFrame
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Alpha/MCMC_alpha_85.csv"
df_MCMC_85 = CSV.read(path, df_MCMC_85)

# Remove burn-in
deleteat!(df_MCMC_85,collect(1:500))

 # Storage
VAF_mat = Array{Float64, 2}(undef, length(pDays), length(df_MCMC_85[:,"iteration"]))
VAF_mat_points = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC_85[:,"iteration"]))
p_mat = Array{Float64, 3}(undef, length(pDays), length(df_MCMC_85[:,"iteration"]), 8)

# Calculate VAF in loop
@showprogress dt=1 desc="Calculating VAF for MCMC chain..." for k in eachindex(df_MCMC_85.rho1)
    # Set rho values
    rho = [df_MCMC_85.rho1[k], df_MCMC_85.rho2[k], df_MCMC_85.alpha[k]]

    # Set initial JAK
    initJAK = df_MCMC_85.initJAK[k]

    # Calculate VAF using function
    VAF_mat[:,k], sol = model_calc_VAF_alpha(rho,df_p,effect,pDays,p,master_curve,initJAK)
    VAF_mat_points[:,k], sol_points = model_calc_VAF_alpha(rho,df_p,effect,pDays2,p,master_curve,initJAK)
    p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
               a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
    p_mat[:,k,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
end
VAF_mat_plot = VAF_mat[1:30:end,:]

In [ ]:
# Calculate RMSE based on mean at the points
RMSE = rmsd(median(VAF_mat_points, dims=2),df_p.JAK[.!isnan.(df_p.JAK)])

# Calculate mean width of 95% CI
mean_width = mean(abs.(quantile.(eachrow(VAF_mat), 0.975)*100-quantile.(eachrow(VAF_mat), 0.025)*100))

# Calculate mean parameter values
median_param_1 = median(df_MCMC_85[:,"rho1"])
median_param_2 = median(df_MCMC_85[:,"rho2"])
median_alpha = median(df_MCMC_85[:,"alpha"])

# Plot results
figVAF = plot()
xlims!(0, 75)
ylims!(0, 100)
# ylabel!(L"JAK2"*" VAF [%]")
annotate!(50, 90, text(L"\textrm{RMSE}=" * "$(round(RMSE; sigdigits=3))", :center, 15))
annotate!(50, 80, text(L"\rho="*"$(round(median_param_1; sigdigits=3))", :center, 15))
annotate!(50, 70, text(L"\delta="*"$(round(median_param_2; sigdigits=3))", :center, 15))
annotate!(50, 60, text(L"\alpha="*"$(round(median_alpha; sigdigits=3))", :center, 15))
title!("Patient $(pID)")
p2 = twinx()
xlims!(p2, 0, 75)
ylims!(p2, 0, 20)
plot!(p2, df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
        linewidth = 3, label = "", colour= :blue1)
plot!(p2, [maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
      linewidth = 3, label = "", colour= :blue1)
#         title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho="
#                *"$(round(median_param_1; sigdigits=3)), " * L"\delta="*"$(round(median_param_2; sigdigits=3))")
# xlabel!(p2, L"t"*" [months]")
# ylabel!(p2, "IFN"*L"-$\alpha$ $\left[\mathrm{\mu g/day}\right]$")
plot!(pDays_plot/30.4, quantile.(eachrow(VAF_mat_plot), 0.025)*100, fillrange = quantile.(eachrow(VAF_mat_plot), 0.975)*100,
      fillalpha = 0.35, colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
plot!(pDays_plot[pDays_plot.<=maxDays]/30.4, median(VAF_mat_plot[pDays_plot.<=maxDays,:],dims=2)*100, linewidth = 3, 
  label = L"JAK2"*" VAF - Median", colour = :black)
plot!(pDays_plot[pDays_plot.>maxDays]/30.4, median(VAF_mat_plot[pDays_plot.>maxDays,:],dims=2)*100, linewidth = 3, 
  label = L"JAK2"*" VAF - Median prediction", colour = :black, linestyle = :dash)
scatter!(pDays2./30.4, pVAF2*100, label=L"\textrm{Data}", markercolor=:red, markersize=6, markerstrokewidth=0.12)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
figVAF_85 = plot(figVAF, size = (700, 500), margin=5mm, leftmargin = 8mm, rightmargin = 8mm)
xlims!(p2, 0, 75)
ylims!(p2, 0, 20)

In [ ]:
 # Plot density
l = @layout [a a; a a; a a; a a; a a;]
plot_rho1_chain = plot(df_MCMC_85.iteration,df_MCMC_85.rho1,linewidth = 0.5, colour = 1, label = "")
title!("\nChain for "*L"\rho")
xlabel!("Iteration")
ylabel!(L"\rho\quad \left[\mathrm{day/\mu g}\right]")
plot_rho1_density = plot(density(df_MCMC_85.rho1,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!("\nDensity for "*L"\rho")
xlabel!(L"\rho\quad \left[\mathrm{day/\mu g}\right]")
ylabel!("pdf")
plot_rho2_chain = plot(df_MCMC_85.iteration,df_MCMC_85.rho2,linewidth = 0.5, colour = 1, label = "")
title!("Chain for "*L"\delta")
xlabel!("Iteration")
ylabel!(L"\delta\quad\left[\mathrm{day/\mu g}\right]")
plot_rho2_density = plot(density(df_MCMC_85.rho2,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!("Density for "*L"\delta")
xlims!(0,1.1)
xlabel!(L"\delta\quad \left[\mathrm{day/\mu g}\right]")
ylabel!("pdf")
plot_alpha_chain = plot(df_MCMC_85.iteration,df_MCMC_85.alpha,linewidth = 0.5, colour = 1, label = "")
title!(L"Chain for $\alpha$")
xlabel!("Iteration")
ylabel!(L"$\alpha$")
plot_alpha_density = plot(density(df_MCMC_85.alpha,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!(L"Density for $\alpha$")
xlabel!(L"$\alpha$")
ylabel!("pdf")
plot_initJAK_chain = plot(df_MCMC_85.iteration,df_MCMC_85.initJAK*100,linewidth = 0.5, colour = 1, label = "")
title!(L"Chain for $v_0$")
xlabel!("Iteration")
ylabel!(L"$v_0$ [%]")
plot_initJAK_density = plot(density(df_MCMC_85.initJAK*100,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 100)
title!(L"Density for $v_0$")
xlabel!(L"$v_0$ [%]")
ylabel!("pdf")
plot_sigma_chain = plot(df_MCMC_85.iteration,df_MCMC_85.sigma*100, linewidth = 0.5, colour = 1, label = "")
title!("Chain for "*L"\sigma_v")
xlabel!("Iteration")
ylabel!(L"$\sigma_v$ [%]")
plot_sigma_density = plot(density((df_MCMC_85.sigma*100),legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 60)
title!("Density for "*L"\sigma_v")
xlims!(0,100)
xlabel!(L"$\sigma_v$ [%]")
ylabel!("pdf")
figMCMC_85 = plot(plot_rho1_chain,plot_rho1_density,plot_rho2_chain,plot_rho2_density,plot_alpha_chain,plot_alpha_density,
    plot_initJAK_chain,plot_initJAK_density,
    plot_sigma_chain,plot_sigma_density,layout = l,size = (700, 1000), margin=5mm, leftmargin = 8mm, rightmargin = 8mm)
        

In [ ]:
pID = 121
df_p = df_D[df_D.patientID .== pID, :]

# Extract latest data point
maxDays = maximum(df_p.days)

# Extract VAF
pVAF = df_p.JAK

# Use only data at which VAF is not NaN
pVAF = pVAF[.!isnan.(pVAF)]


# Choose treatment effect
effect = "py0dy1"

# Chooase to estimate initJAK as well
estInitJAK = true
    
# Instantiate model
model = model_infer_dynamics_alpha(pVAF,df_p,effect,p,master_curve,estInitJAK)

# Set seed 
Random.seed!(42)

# Sample using Turing
MCMC = sample(model, HMC(0.05, 10), 10500)

# Convert to data frame
df_MCMC_121 = DataFrame(MCMC)

In [ ]:
# Save results
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Alpha/MCMC_alpha_121.csv"
CSV.write(path, df_MCMC_121)

In [ ]:
# Extract data
pID = 121
df_p = df_D[df_D.patientID .== pID, :]

# Choose time for calculating VAF
pDays = 0:1:75*30.4
pDays2 = df_p.days[.!isnan.(df_p.JAK)]
pVAF2 = df_p.JAK[.!isnan.(df_p.JAK)]
pDays_plot = pDays[1:30:end]

# Choose treatment effect
effect = "py0dy1"

# Extract latest data point
maxDays = maximum(df_p.days)

# Load results
df_MCMC_121 = DataFrame
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Alpha/MCMC_alpha_121.csv"
df_MCMC_121 = CSV.read(path, df_MCMC_121)

# Remove burn-in
deleteat!(df_MCMC_121,collect(1:500))

 # Storage
VAF_mat = Array{Float64, 2}(undef, length(pDays), length(df_MCMC_121[:,"iteration"]))
VAF_mat_points = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC_121[:,"iteration"]))
p_mat = Array{Float64, 3}(undef, length(pDays), length(df_MCMC_121[:,"iteration"]), 8)

# Calculate VAF in loop
@showprogress dt=1 desc="Calculating VAF for MCMC chain..." for k in eachindex(df_MCMC_121.rho1)
    # Set rho values
    rho = [df_MCMC_121.rho1[k], df_MCMC_121.rho2[k], df_MCMC_121.alpha[k]]

    # Set initial JAK
    initJAK = df_MCMC_121.initJAK[k]

    # Calculate VAF using function
    VAF_mat[:,k], sol = model_calc_VAF_alpha(rho,df_p,effect,pDays,p,master_curve,initJAK)
    VAF_mat_points[:,k], sol_points = model_calc_VAF_alpha(rho,df_p,effect,pDays2,p,master_curve,initJAK)
    p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
               a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
    p_mat[:,k,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
end
VAF_mat_plot = VAF_mat[1:30:end,:]

In [ ]:
# Calculate RMSE based on mean at the points
RMSE = rmsd(median(VAF_mat_points, dims=2),df_p.JAK[.!isnan.(df_p.JAK)])

# Calculate mean width of 95% CI
mean_width = mean(abs.(quantile.(eachrow(VAF_mat), 0.975)*100-quantile.(eachrow(VAF_mat), 0.025)*100))

# Calculate mean parameter values
median_param_1 = median(df_MCMC_121[:,"rho1"])
median_param_2 = median(df_MCMC_121[:,"rho2"])
median_alpha = median(df_MCMC_121[:,"alpha"])

# Plot results
figVAF = plot()
xlims!(0, 75)
ylims!(0, 100)
# ylabel!(L"JAK2"*" VAF [%]")
annotate!(50, 90, text(L"\textrm{RMSE}=" * "$(round(RMSE; sigdigits=3))", :center, 15))
annotate!(50, 80, text(L"\rho="*"$(round(median_param_1; sigdigits=3))", :center, 15))
annotate!(50, 70, text(L"\delta="*"$(round(median_param_2; sigdigits=3))", :center, 15))
annotate!(50, 60, text(L"\alpha="*"$(round(median_alpha; sigdigits=3))", :center, 15))
title!("Patient $(pID)")
p2 = twinx()
xlims!(p2, 0, 75)
ylims!(p2, 0, 20)
plot!(p2, df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
        linewidth = 3, label = "", colour= :blue1)
plot!(p2, [maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
      linewidth = 3, label = "", colour= :blue1)
#         title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho="
#                *"$(round(median_param_1; sigdigits=3)), " * L"\delta="*"$(round(median_param_2; sigdigits=3))")
# xlabel!(p2, L"t"*" [months]")
# ylabel!(p2, "IFN"*L"-$\alpha$ $\left[\mathrm{\mu g/day}\right]$")
plot!(pDays_plot/30.4, quantile.(eachrow(VAF_mat_plot), 0.025)*100, fillrange = quantile.(eachrow(VAF_mat_plot), 0.975)*100,
      fillalpha = 0.35, colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
plot!(pDays_plot[pDays_plot.<=maxDays]/30.4, median(VAF_mat_plot[pDays_plot.<=maxDays,:],dims=2)*100, linewidth = 3, 
  label = L"JAK2"*" VAF - Median", colour = :black)
plot!(pDays_plot[pDays_plot.>maxDays]/30.4, median(VAF_mat_plot[pDays_plot.>maxDays,:],dims=2)*100, linewidth = 3, 
  label = L"JAK2"*" VAF - Median prediction", colour = :black, linestyle = :dash)
scatter!(pDays2./30.4, pVAF2*100, label=L"\textrm{Data}", markercolor=:red, markersize=6, markerstrokewidth=0.12)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
figVAF_121 = plot(figVAF, size = (700, 500), margin=5mm, leftmargin = 8mm, rightmargin = 8mm)
xlims!(p2, 0, 75)
ylims!(p2, 0, 20)

In [ ]:
 # Plot density
l = @layout [a a; a a; a a; a a; a a;]
plot_rho1_chain = plot(df_MCMC_121.iteration,df_MCMC_121.rho1,linewidth = 0.5, colour = 1, label = "")
title!("\nChain for "*L"\rho")
xlabel!("Iteration")
ylabel!(L"\rho\quad \left[\mathrm{day/\mu g}\right]")
plot_rho1_density = plot(density(df_MCMC_121.rho1,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!("\nDensity for "*L"\rho")
xlabel!(L"\rho\quad \left[\mathrm{day/\mu g}\right]")
ylabel!("pdf")
plot_rho2_chain = plot(df_MCMC_121.iteration,df_MCMC_121.rho2,linewidth = 0.5, colour = 1, label = "")
title!("Chain for "*L"\delta")
xlabel!("Iteration")
ylabel!(L"\delta\quad\left[\mathrm{day/\mu g}\right]")
plot_rho2_density = plot(density(df_MCMC_121.rho2,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!("Density for "*L"\delta")
xlims!(0,1.1)
xlabel!(L"\delta\quad \left[\mathrm{day/\mu g}\right]")
ylabel!("pdf")
plot_alpha_chain = plot(df_MCMC_121.iteration,df_MCMC_121.alpha,linewidth = 0.5, colour = 1, label = "")
title!(L"Chain for $\alpha$")
xlabel!("Iteration")
ylabel!(L"$\alpha$")
plot_alpha_density = plot(density(df_MCMC_121.alpha,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 1)
title!(L"Density for $\alpha$")
xlabel!(L"$\alpha$")
ylabel!("pdf")
plot_initJAK_chain = plot(df_MCMC_121.iteration,df_MCMC_121.initJAK*100,linewidth = 0.5, colour = 1, label = "")
title!(L"Chain for $v_0$")
xlabel!("Iteration")
ylabel!(L"$v_0$ [%]")
plot_initJAK_density = plot(density(df_MCMC_121.initJAK*100,legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 100)
title!(L"Density for $v_0$")
xlabel!(L"$v_0$ [%]")
ylabel!("pdf")
plot_sigma_chain = plot(df_MCMC_121.iteration,df_MCMC_121.sigma*100, linewidth = 0.5, colour = 1, label = "")
title!("Chain for "*L"\sigma_v")
xlabel!("Iteration")
ylabel!(L"$\sigma_v$ [%]")
plot_sigma_density = plot(density((df_MCMC_121.sigma*100),legend=false),linewidth = 0.5, colour = 1, label = "")
xlims!(0, 60)
title!("Density for "*L"\sigma_v")
xlims!(0,100)
xlabel!(L"$\sigma_v$ [%]")
ylabel!("pdf")
figMCMC_121 = plot(plot_rho1_chain,plot_rho1_density,plot_rho2_chain,plot_rho2_density,plot_alpha_chain,plot_alpha_density,
    plot_initJAK_chain,plot_initJAK_density,
    plot_sigma_chain,plot_sigma_density,layout = l,size = (700, 1000), margin=5mm, leftmargin = 8mm, rightmargin = 8mm)
        

In [ ]:
# Plot VAF together
l = @layout [a a a]
fig_bar_time = plot(figVAF_3, figVAF_38, figVAF_121, layout = l, size = (1500, 500), margin=5mm, leftmargin = 8mm, 
                    rightmargin = 8mm, bottommargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_bar_time,figpath*"zygosity_VAF.png")
savefig(fig_bar_time,figpath*"zygosity_VAF.pdf")
savefig(fig_bar_time,figpath*"zygosity_VAF.svg")

In [ ]:
# Plot MCMC together
l = @layout [a a a]
fig_bar_time = plot(figMCMC_3, figMCMC_38, figMCMC_121, layout = l, size = (2000, 800), margin=5mm, leftmargin = 8mm, 
                    rightmargin = 8mm, bottommargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/"
savefig(fig_bar_time,figpath*"zygosity_MCMC.png")
savefig(fig_bar_time,figpath*"zygosity_MCMC.pdf")
savefig(fig_bar_time,figpath*"zygosity_MCMC.svg")